In [1]:
import os
import numpy as np
import pandas as pd
from scipy.stats import kurtosis
from joblib import load

from scipy.signal import butter, filtfilt, find_peaks, peak_prominences
from sklearn.preprocessing import MinMaxScaler

**Daten laden**<br>
Rohdaten werden aus einem Verzeichnis geladen und einem Dictionary data hinzugefügt.<br>
Sollten Keine csv-Daten vorhanden sein bleibt das Dictionary data leer.

In [2]:
# Laden aller Daten aus dem Ordner: 'csv_data'
def load_csv_data(folder_name):
    data = {}
    csv_folder = os.path.join(os.getcwd(), folder_name)

    if not os.path.exists(csv_folder):
        print(f"Error: The folder '{csv_folder}' was not found.")
        return data
    
    for file in os.listdir(csv_folder):
        if file.endswith('.csv'):
            file_path = os.path.join(csv_folder, file)
            try:
                filename = file.replace('.csv', '')
                data[filename] = pd.read_csv(file_path, delimiter=',')
            except Exception as e:
                print(f"Error reading {file}: {e}")

    return data


Die Funktion load_csv_data wird mit dem Parameter des Verzeichnises in dem sich die csv-Daten befinden aufgerufen und der Variable data zugewiesen.<br>
Danach wird versucht die jeweiligen Zeilen Gyroscope und Linear_Acceleration aus den jeweiligen Sprungdaten und Kniebeugendaten zu lesen.<br>
Sollte einer der Schlüssel nicht gefunden werden wird ein KeyError geworfen.

In [3]:
def unpack_data_train(folder_name):
    data = load_csv_data(folder_name)
    print(data.keys())

    try:
        jump_acc = data['Linear_Acceleration_Jump']
        jump_gyr = data['Gyroscope_Jump']
        squat_acc = data['Linear_Acceleration_Squat']
        squat_gyr = data['Gyroscope_Squat']
    except KeyError as e:
        print(f"Key not found: {e}")

    return jump_acc, jump_gyr, squat_acc, squat_gyr

In [4]:
def unpack_data_test(folder_name):
    data = load_csv_data(folder_name)
    print(data.keys())

    try:
        gyr = data['Gyroscope']
        acc = data['Linear Acceleration']
    except KeyError as e:
        print(f"Key not found: {e}")

    return acc, gyr

Die Funktion trim_dataframes_to_shortest kürzt alle übergebenen Dataframes auf die Länge des kürzesten Dataframes.<br>
Es wird die minimale Länge aller Dataframes bestimmt und anschließend werden alle Dataframes auf die minimale Länge gekürzt.

In [5]:
def trim_dataframes_to_shortest(df_list):
    # Bestimmen der minimalen Länge aller DataFrames in der Liste
    min_length = min(df.shape[0] for df in df_list)

    # Kürzen aller DataFrames auf die minimale Länge
    trimmed_dfs = [df.iloc[:min_length] for df in df_list]

    return trimmed_dfs

Die Funktion combine_relevant_columns erhält zwei Dataframes und erstellt ein neues Dataframe das nur bestimmte relevante Daten enthält.<br>
Es werden die Spalten für die Lineare Beschleunigung auf der z und y Achse sowie die x Achse und die Zeit des Gyroscopes gelesen.<br>
Am Ende wird ein neues Dataframe ausgegeben mit den gelesenen Spalten.<br>

In [6]:
def combine_relevant_columns(acc_df, gyr_df):
    linear_acc_z = acc_df['Linear Acceleration z (m/s^2)']
    linear_acc_y = acc_df['Linear Acceleration y (m/s^2)']

    gyroskop_x = gyr_df['Gyroscope x (rad/s)']

    time = gyr_df['Time (s)']

    return pd.DataFrame({
        'Time (s)': time,
        'Linear Acceleration z (m/s^2)': linear_acc_z,
        'Linear Acceleration y (m/s^2)': linear_acc_y,
        'Gyroscope x (rad/s)': gyroskop_x
    })

Die Funktion filter_frequency führt einen Butterworth-Tiefpassfilter auf das mitgegebene Dataframe durch.<br>
Es wird eine Kopie des übergebenen Dataframes erstellt um die gefilterten Daten zu speichern.<br>
Die Filterordnung wird auf 5 festgelegt und der low_cutoff Frequenz wird auf 5Hz festgelegt.<br>
Als nächstes wird der Butterworth Tiefpassfilter erstellt mit b und a als Koeffizienten.<br>
Es wird über alle Spalten außer Time (s) mit einer Schleife iteriert und der Filter wird auf die jeweiligen Spalten angewendet und in das neue Kopierte Dataframe gespeichert.<br>
Abschließend wird das Dataframe mit den neuen gefilterten Daten zurückgegeben.

In [7]:
def filter_frequency(dataframe):
    # Kopie des DataFrames erstellen
    filtered_dataframe = dataframe.copy()

    # Festlegen der Cutoff-Frequenz und der Filterordnung
    filter_order = 5
    low_cutoff = 0.3
    high_cutoff = 10

    # Butterworth Tiefpass erstellen
    b, a = butter(filter_order, [low_cutoff,high_cutoff], btype='band', analog=False, fs=100)

    # Schleife über alle Spalten außer 'Time (s)'
    for column in dataframe.columns:
        if column != 'Time (s)':
            # Hochpassfilter auf die Signalreihe anwenden
            filtered_signal = filtfilt(b, a, dataframe[column])
            filtered_dataframe[column] = filtered_signal

    return filtered_dataframe

**Code zur Normalisierung**<br>
Um unsere Daten vorzubereiten, verwenden wir eine Normalisierungsfunktion, die die Merkmale 'Linear Acceleration z (m/s^2)',<br>
 'Linear Acceleration y (m/s^2)' und 'Gyroscope x (rad/s)' skaliert. 

In [8]:
def normalize_df(df):
    # Kopie des DataFrames erstellen
    normalized_df = df.copy()

    columns_to_scale = ['Linear Acceleration z (m/s^2)', 'Linear Acceleration y (m/s^2)', 'Gyroscope x (rad/s)']

    # Initialisieren des MinMaxScaler mit einem Bereich von -1 bis 1
    scaler = MinMaxScaler(feature_range=(-1, 1))

    # Fit des Scalers auf die Daten
    scaler.fit(normalized_df[columns_to_scale])

    # Transformieren der Daten
    normalized_columns = scaler.transform(normalized_df[columns_to_scale])
    
    # Ersetzen der Originaldaten durch die normalisierten Daten
    normalized_df[columns_to_scale] = normalized_columns

    return normalized_df

**Windowing**<br>
Erstellen der Windows, mit Größe 100 und Overlap von 50. Der dynamische Windowing-Ansatz hat bei uns zu unakuraten<br> Ergebnissen
im Entscheidungsbaum gesorgt, daher haben wir uns für die statische Variante entschieden.

In [9]:
def create_windows(df):
    windows = []
    window_size = 100
    overlap = 50

    for i in range(0, len(df), window_size - overlap):
        window = df.iloc[i:i+window_size]
        windows.append(window)

    return windows

**Labeling**
Auf Basis der Windowgrößen werden  die Labels erstellt

In [10]:
def create_labels(df):
    labels = []
    window_size = 100
    overlap = 50

    for i in range(0, len(df), window_size - overlap):
        window = df.iloc[i:i+window_size]
        labels.append(window.iloc[0]['Label'])

    return labels

**Zusamenfügen der Dataframes**<br>
Mit dem Addieren von `max_time_squat`auf die Zeit von`df2` wird eine kontinuierlicher Graph ohne Überscheidungen erzeugt

In [11]:
def combine_dataframes(df1, df2):
    max_time_squat = df1['Time (s)'].max()
    df2['Time (s)'] = df2['Time (s)'] + max_time_squat
    return pd.concat([df1, df2], ignore_index=True)

**Feature-Engineering**<br>
Die Funktion `calculate_peak_features` extrahiert relevante statistische Daten über Peaks der y-Beschleunigungskomponente

In [12]:
def calculate_peak_features(window):
    kurt = kurtosis(window['Linear Acceleration y (m/s^2)'])
 
    peaks, _ = find_peaks(window['Linear Acceleration y (m/s^2)'])
    num_peaks = len(peaks)
    
    prominences = peak_prominences(window['Linear Acceleration y (m/s^2)'], peaks)[0]
    avg_prominence = np.mean(prominences) if prominences.size > 0 else 0
    
    return kurt, num_peaks, avg_prominence

Hier werden alle relevanten Features zum Training der Daten berechnet. Dazu werden die Daten zunächst in Fenster der 
Länge 100 mit einer Überlappung von 50 aufgeteilt. Für jedes Fenster werden dann die Features berechnet. 

In [13]:
def create_feature_df(windows):
    feature_list = []

    for window in windows:
        acc_y = window['Linear Acceleration y (m/s^2)']
        acc_z = window['Linear Acceleration z (m/s^2)']
        gyr_x = window['Gyroscope x (rad/s)']
        
        gyr_x_var = gyr_x.var()
        mean_acc_z = acc_z.mean()
        mean_acc_y = acc_y.mean()
        max_acc_y = acc_y.max()
        quantile_75_acc_y = acc_y.quantile(0.75)
        quartile_25_acc_y = acc_y.quantile(0.25)
        std_acc_z = acc_z.std()
        var_acc_z = acc_z.var()
        min_acc_z = acc_z.min()
        max_acc_z = acc_z.max()
        range_acc_z = max_acc_z - min_acc_z
        peak_acc_y = acc_y.max()
        std_acc_y = acc_y.std()
        var_acc_y = acc_y.var()
        min_acc_y = acc_y.min()
        range_acc_y = peak_acc_y - min_acc_y
        range_gyr_x = gyr_x.max() - gyr_x.min()
        range_acc_z = acc_z.max() - acc_z.min()
        min_acc_y = acc_y.min()


        kurt, num_peaks, avg_prominence = calculate_peak_features(window)


        features = {
            'gyr_x_var': gyr_x_var,
            'avg_prominence': avg_prominence,
            'kurt' : kurt,
            'num_peaks' : num_peaks,
            'min_acc_y': min_acc_y,
            'mean_acc_z': mean_acc_z,
            'quantile_75_acc_y': quantile_75_acc_y,
            'quantile_25_acc_y': quartile_25_acc_y,
            'peak_acc_y': max_acc_y,
            'min_acc_z': min_acc_z,
            'peak_acc_z': max_acc_z,
            'range_gyr_x': range_gyr_x,
            'range_acc_z': range_acc_z,
            'var_acc_z': var_acc_z,
            'std_acc_z': std_acc_z,
            'mean_acc_y': mean_acc_y,
            'std_acc_y': std_acc_y,
            'var_acc_y': var_acc_y,
            'range_acc_y': range_acc_y,
            'min_acc_y': min_acc_y
        }
        feature_list.append(features)

    return pd.DataFrame(feature_list)


In [14]:
def load_model(pfad):
    try:
        modell = load(pfad)
        print(f"Modell erfolgreich von '{pfad}' geladen.")
        return modell
    except FileNotFoundError:
        print(f"Modell konnte nicht gefunden werden: '{pfad}'")
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten beim Laden des Modells: {e}")